In [1]:
import pandas as pd
import statsmodels.api as sm

<h1>Efects de la asistencia en el examen final</h1>
<p style="font-size:20px">

La base de datos ATTEND contiene información sobre 680 estudiantes de un curso de microeconomía.
A continuación se describen las variables disponibles:
- attend: número de clases asistidas (de un total de 32).
- priGPA: GPA acumulado previo al curso.
- ACT: puntaje de admisión estandarizado ACT.
- atndrte: porcentaje de clases asistidas (entre 0 y 100).
- frosh: indicador (=1) si el estudiante es de primer a no (freshman).
- soph: indicador (=1) si el estudiante es de segundo a no (sophomore).
- skipped: número de clases que el estudiante se perdi’o.
- stndfnl: nota estandarizada del examen final .

Un modelo para explicar el resultado estandarizado de un examen final (stndfnl) en términos del
porcentaje de clases asistidas (atndrte), el promedio de calificaciones de la universidad previa y
la puntuación del ACT es:
$$
stndfnl =β_0 + β_1atndrte + β_2priGPA + β_3ACT + β_4priGPA^{2}
+ β_5ACT^{2} + β_6priGPA · atndrte + e
$$

In [2]:
url=r'C:\Users\Camil\Documents\ESPOL\ECOP2037_CE\hmw1\attend.dta'
df=pd.read_stata(url)

1. ¿Por qué usamos el resultado estadarizado en lugar de la nota original?

Una de las razones para usar el resultado estandarizado es que las notas originales pueden estar en diferentes escalas o tener distribuciones distintas segun otras caracteristicas. Al estandarizar, todas las observaciones pasan a una escala común con media 0 y desviación estándar 1, haciendolas comparables entre si. 

2. ¿Por qué incluimos priGPA^2 , ACT^2 , y priGPA · atndrte?

Incluimos las variables priGPA y ACT en su forma cuadrática porque nos permite capturar relaciones no lineales. Al elevar al cuadrado, permitimos que el modelo se pueda adaptar a una curva y no solo a una línea recta. Entonces, por ejemplo, si priGPA tiene rendimientos decrecientes sobre stndfnl, esta forma cuadrática lo podrá capturar en el modelo.


Incluimos la interacción priGPA · atndrte porque el efecto de asistir a clases puede cambiar según el promedio anterior del estudiante. Así, el modelo reconoce que no todos se benefician igual de las clases asistidas.


3. Estime el modelo propuesto. Reporte los coeficientes estimados y sus errores estándar

In [3]:
##creamos las versiones no lineales de las variables
df['priGPA2'] = df['priGPA']**2
df['ACT2'] = df['ACT']**2
df['priGPA_atndrte'] = df['priGPA'] * df['atndrte']

In [4]:
##Estimamos el modelo
X = df[['atndrte','priGPA','ACT','priGPA2','ACT2','priGPA_atndrte']]
X = sm.add_constant(X)  
y = df['stndfnl']
model = sm.OLS(y, X).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                stndfnl   R-squared:                       0.229
Model:                            OLS   Adj. R-squared:                  0.222
Method:                 Least Squares   F-statistic:                     33.25
Date:                Fri, 30 May 2025   Prob (F-statistic):           3.49e-35
Time:                        01:26:55   Log-Likelihood:                -868.90
No. Observations:                 680   AIC:                             1752.
Df Residuals:                     673   BIC:                             1783.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
const              2.0503      1.360      1.

4. ¿Por qué no se debe interpretar directamente el coeficiente de atndrte en presencia de la
interacción con priGPA? ¿A qué valor de priGPA corresponde esta interpretación y por qué
es poco relevante en la muestra?

Al incluir dentro del modelo la interacción de la variable atndrte y priGPA, el coeficiente de atndrte (es decir $β_1$) va a representar el efecto de atender a clases (atndrte) en la nota final estandarizada (stndnl) cuando priGPA es igual a cero. Dado que en la base de datos no hay alguna observación cuyo valor de priGPA sea igual a cero entonces interpretar unicamente $β_1$ no es relevante.

5. Calcule el efecto marginal de atndrte sobre stndfnl cuando priGPA = 2.59 (la media).
Interprete el resultado en términos de desviaciones estandarizadas.

El efecto marginal es: $\frac{\partial\, stndfnl}{\partial\, atndrte} = \beta_1 + \beta_6 \cdot priGPA$
donde:
- β_1= -0.0067
- β_6=  0.0056
- priGPA= 2.59

In [5]:
b1 = model.params['atndrte']
b6 = model.params['priGPA_atndrte']

efecto=(b1)+[(2.59)*b6]
print(efecto)

[0.00775457]


Cuando priGPA = 2.59, un aumento de una unidad en la tasa de asistencia (atndrte) está asociado con un aumento de 0.0078 desviaciones estandar en la nota final (stndfnl), manteniendo todo lo demás constante.

6. Centre la variable priGPA alrededor de su media y reemplace priGPA·atndrte por (priGPA−
2,59) · atndrte. Estime este modelo nuevamente y reporte el nuevo coeficiente de atndrte.
Especifacmente el modelo a estimar es
$$stndfnl =β0 + β1atndrte + β2priGPA + β3ACT + β4priGPA2+ β5ACT2 + β6(priGPA − 2,59) · atndrte + e$$
Compare con el obtenido previamente.

In [6]:
##centramos y creamos nueva var interaccion
df['priGPA_c'] = df['priGPA'] - 2.59
df['priGPA_c_atndrte'] = df['priGPA_c'] * df['atndrte']

In [7]:
##estimamos el modelo nuevamente
X = df[['atndrte','priGPA','ACT','priGPA2','ACT2','priGPA_c_atndrte']]
X = sm.add_constant(X)  
y = df['stndfnl']
model2 = sm.OLS(y, X).fit()
print(model2.summary())

                            OLS Regression Results                            
Dep. Variable:                stndfnl   R-squared:                       0.229
Model:                            OLS   Adj. R-squared:                  0.222
Method:                 Least Squares   F-statistic:                     33.25
Date:                Fri, 30 May 2025   Prob (F-statistic):           3.49e-35
Time:                        01:26:55   Log-Likelihood:                -868.90
No. Observations:                 680   AIC:                             1752.
Df Residuals:                     673   BIC:                             1783.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
const                2.0503      1.360  

En este caso, el B_1 que en el modelo anterior representaba el efecto de atndrte en stndfnl cuando priGPA=0, ahora representa este mismo efecto pero cuando priGPA es igual a su media 2.59. 

7. Derive el efecto marginal de priGPA sobre stndfnl. Evalúelo cuando priGPA = 2.59 y
atndrte = 82. Interprete este resultado.

El efecto marginal es: $\frac{\partial\, stndfnl}{\partial\, priGPA} = \beta_2 +2 \beta_4 \cdot priGPA+ \beta_6 \cdot atndrte$
donde:
- β_2= -1.6285
- β_6=  0.0056
- β_4=  0.2959
- priGPA= 2.59
- atndrte= 82


In [8]:
b2 = model2.params['priGPA']
b4 = model2.params['priGPA2']
b6 = model2.params['priGPA_c_atndrte'] 

priGPA = 2.59
atndrte = 82

efecto2 = b2 + 2 * b4 * priGPA + b6 * atndrte
print(efecto2)

0.3622902438547918


Cuando priGPA = 2.59 y atndrte = 82, es decir su promedio, un aumento de una unidad en la nota previa se asocia con un aumento de aproximadamente 0.3643 desviaciones estándar en la nota final estandarizada (stndfnl), manteniendo todo el resto constante.

8. Centre simultáneamente priGPA y atndrte en sus medias. Especificamente el modelo a estimar es
$$stndfnl =β_0 + β_1atndrte + β_2priGPA + β_3ACT + β_4(priGPA − 2,59)^{2} + β_5ACT^{2} + β_6priGPA · (atndrte − 82) + e$$
¿Cómo se interpreta ahora el coeficiente de priGPA? Este resultado se conoce como el efecto
parcial promedio (APE) de priGPA sobre stndfnl.

In [9]:
##centramos y creamos nueva var interaccion
df['atndrte_c'] = df['atndrte'] - 82
df['priGPA_c_atndrte_c'] = df['priGPA_c'] * df['atndrte_c']

In [10]:
##estimamos el modelo nuevamente
X = df[['atndrte','priGPA','ACT','priGPA2','ACT2','priGPA_c_atndrte_c']]
X = sm.add_constant(X)  
y = df['stndfnl']
model3 = sm.OLS(y, X).fit()
print(model3.summary())

                            OLS Regression Results                            
Dep. Variable:                stndfnl   R-squared:                       0.229
Model:                            OLS   Adj. R-squared:                  0.222
Method:                 Least Squares   F-statistic:                     33.25
Date:                Fri, 30 May 2025   Prob (F-statistic):           3.49e-35
Time:                        01:26:55   Log-Likelihood:                -868.90
No. Observations:                 680   AIC:                             1752.
Df Residuals:                     673   BIC:                             1783.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
const                  0.8640      1

$β_2$ es el efecto parcial promedio (APE) de priGPA sobre stndfnl: mide cuánto cambia stndfnl (en desviaciones estándar) ante un aumento de una unidad en priGPA, en promedio, manteniendo el resto constante y cuando atndrte = 82.

9. Explique por que los efectos marginales en este modelo dependen de los valores de las variables explicativas. ¿Por que los APEs son una forma resumida de interpretar estos efectos?

En el modelo, los efectos marginales dependen de los valores de las variables explicativas lo que significa que el impacto de una variable sobre stndfnl varía según el nivel de otras variables.
Los efectos parciales promedio son útiles porque resumen esa variación en un solo valor representativo, evaluando el efecto en un caso promedio.


10. Una visión teórica: Suponga el modelo
$$y = β_0 + β_1x_1 + β_2x_2 + β_3x_1x_2 + e$$
Reescríbalo como
$$y = α_0 + δ_1x_1 + δ_2x_2 + β_3(x_1 − µ_1)(x_2 − µ_2) + e$$

Muestre que $δ_2 = β_2+β_3µ_1$ y explique por qué esta reparametrización es útil para interpretar
efectos parciales en el promedio. ¿Cómo interpretaría ahora los coeficientes sobre x1 y x2 en
este nuevo modelo?¿En vez de µ1y µ2 puede usar otros valores de la distribución de x1 y x2?


$$x_1 x_2= (x_1 - µ_1 +µ_1) (x_2 - µ_2 +µ_2) $$
$$ = (x_1 - µ_1) (x_2 - µ_2) +u_2(x_1 + µ_1) + u_1(x_2 + µ_2) + µ_1 µ_2 $$
$$ (x_1 - µ_1) (x_2 - µ_2) +µ_2 x_1 +µ_1 x_2 + µ_1 µ_2 $$

Reemplazamos esto en el modelo original
$$y = β_0 + β_1x_1 + β_2x_2 + β_3[(x_1 - µ_1) (x_2 - µ_2) +µ_2 x_1 +µ_1 x_2 + µ_1 µ_2  ] + e$$
$$y = β_0 + β_3 (µ_1 µ_2) + β_1x_1 +β_3 (µ_2 x_1) + β_2x_2 + β_3(µ_1 x_2) + β_3[(x_1 - µ_1) (x_2 - µ_2)] + e$$
$$y = β_0 + β_3 (µ_1 µ_2) + x_1(β_1 +β_3 µ_2) +x_2 (β_2 + β_3µ_1) + β_3[(x_1 + µ_1) (x_2 + µ_2)] + e$$

Si
- $α_0= β_0 + β_3 (µ_1 µ_2) $
- $δ_1 = β_1 +β_3 µ_2 $
- $δ_2 = β_2 + β_3µ_1$

Reemplazamos en el  modelo:

$$y = α_0 + δ_1x_1 + δ_2x_2 + β_3(x_1 − µ_1)(x_2 − µ_2) + e$$



- Esta reparametricación es util porque, al centrar la interacción de $x_1$ y $x_2$ en sus medias la interpretación de cada uno de los delta va a representar el efecto de esa variable en el outcome cuando la otra es igual a su promedio. 
- Bajo esta reparametrización $𝛿_1$ es el efecto marginal de $x_1$ sobre y cuando $x_2$ es igual a su media $𝜇_2$. Así mismo,  $𝛿_2$ es el efecto marginal de $x_2$ sobre y cuando $x_1$ es igual a su media $𝜇_1$.
- Si se pueden usar otro valor representativo de la distribución muestra para reparametrizar el modelo, por ejemplo, la mediana.
